In [73]:
# GOOGLE MAPS PHOTO API

import os
import pandas as pd
import requests
base_photo_url = 'https://maps.googleapis.com/maps/api/place/photo?maxwidth=300&photo_reference='
base_url = 'https://maps.googleapis.com/maps/api/place/details/json?'
key = 'xxx'
fields = 'name,photos'
https://maps.googleapis.com/maps/api/place/photo?maxwidth=300&photo_reference=xxxxx&key=xxxxx

directory_path = os.getcwd()
df_details = pd.read_csv(directory_path + '/details.csv')
df_photos = pd.read_csv(directory_path + '/photos.csv')
for index, row in df_details.iterrows():
    name = row['detail_name']
    pub_id = row['pub_identity']
    full_url = base_url + "place_id=" + place_id + "&key=" + key + "&fields=" + fields
    # full_photo_url = base_photo_url + photo_id + "&key=" + key 
    # + "&fields=" + fields

    response = requests.get(full_url)
    photo_ids = response.json()['result']['photos']
    total_photo_list = []
    for x in photo_ids:
        new_list = [x['photo_reference'], 'false', pub_id]
        # print(new_list)
        total_photo_list.append(new_list)
        # print(new_photo_list)
    # print(total_photo_list)
    df_new_photo = pd.DataFrame(total_photo_list, columns=['photo_identity', 'photo_deletion', 'pub_identity'])
    # print(df_new_photo)
    df_photos = pd.concat([df_photos, df_new_photo], ignore_index=True)
    df_photos.to_csv(directory_path + '/photos_api.csv', index=False, sep=',', encoding='utf-8')
    

In [105]:
# GOOGLE MAPS PLACE API

place_id = 'ChIJy2QpfVUDdkgR_ZjHMTwf28A'
base_url = 'https://maps.googleapis.com/maps/api/place/details/json?'
key = 'xxx'
fields = 'address_component'
full_url = base_url + "place_id=" + place_id + "&key=" + key + "&fields=" + fields
response = requests.get(full_url)
print(response.json())

# components_by_type = {};
# for x in response.json()['result']['address_components']:
#     components_by_type[x['types'][0]] = x['short_name']


{'html_attributions': [], 'result': {'address_components': [{'long_name': '29', 'short_name': '29', 'types': ['street_number']}, {'long_name': 'Watling Street', 'short_name': 'Watling St', 'types': ['route']}, {'long_name': 'Greater', 'short_name': 'Greater', 'types': ['route']}, {'long_name': 'London', 'short_name': 'London', 'types': ['postal_town']}, {'long_name': 'Greater London', 'short_name': 'Greater London', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'England', 'short_name': 'England', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United Kingdom', 'short_name': 'GB', 'types': ['country', 'political']}, {'long_name': 'EC4M 9BR', 'short_name': 'EC4M 9BR', 'types': ['postal_code']}]}, 'status': 'OK'}


In [ ]:
# GOOGLE MAPS PLACE API TEXTSEARCH

YOUR_API_KEY = 'xxxxx'
url = "https://maps.googleapis.com/maps/api/place/textsearch/json" 
QUERY2 = 'restaurants%20in%20Sydney'
QUERY = 'bars%20in%20London'
LAT = '51.52391'
LNG = '-0.09207'
REGION = 'gb'
# RADIUS = '5'
url = f"""https://maps.googleapis.com/maps/api/place/textsearch/json?region={REGION}&location={LAT}%2C{LNG}&query={QUERY}&key={YOUR_API_KEY}"""

response = requests.get(url)
print(requests.get(url).text)

PAGE_TOKEN = requests.get(url).json()['next_page_token']

pub_list = []
google_pubs = requests.get(url).json()['results']
for pb in google_pubs:
#     print(pb)
    pub_list.append({
        'detail_latitude': pb['geometry']['location']['lat'], 
        'detail_latitude': pb['geometry']['location']['lng'],
        'address': pb['formatted_address'],
        'detail_name': pb['name'],
        'place': pb['place_id'],
        'category': pb['types'][0],
        'ranking': pb['rating'],
        'url': pb['url'] if 'url' in pb else '',
        'website': pb['website'] if 'website' in pb else ''
    })
# print(pub_list)

dfItem_ggl = pd.DataFrame.from_records(pub_list)
dfItem_ggl
# print(PAGE_TOKEN)

url = f"""https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken={PAGE_TOKEN}&region={REGION}&location={LAT}%2C{LNG}&query={QUERY}&key={YOUR_API_KEY}"""


In [ ]:
# GOOGLE MAPS PYTHON LIBRARY

import googlemaps
import time

k = 'xxxxxxxxx'

gmaps = googlemaps.Client(key=k)

dfItem_ggl = pd.DataFrame(columns=['place','name','address','latitude','longitude','category','ranking','url','website'])
# for y in range (-30, 12, 2):
for y in range (-14, 12, 2):
    # print(y/100)
    
    for x in range(5140, 5174, 2):
        # print(x/100)

        params = {'query': ['pub'], 'location': (x/100, y/100) }
        for ab in range(3):
            time.sleep(2)
            gm = gmaps.places(**params)
            pub_list = []
            for pb in gm['results']:
                pub_list.append({
                    'place': pb['place_id'],
                    'name': pb['name'],
                    'address': pb['formatted_address'],
                    'latitude': pb['geometry']['location']['lat'], 
                    'longitude': pb['geometry']['location']['lng'],                  
                    'category': pb['types'][0],
                    'ranking': pb['rating'],
                    'url': pb['url'] if 'url' in pb else '',
                    'website': pb['website'] if 'website' in pb else ''
                })
            dfItem_ggl_temp = pd.DataFrame.from_records(pub_list)
            dfItem_ggl = pd.concat([dfItem_ggl, dfItem_ggl_temp], ignore_index=True) 
            if 'next_page_token' in gm:
                params['page_token'] = gm['next_page_token'] 
            else:
                break
# dfItem_ggl
dfItem_ggl.to_csv(f'{pathway}/files/dfItem_ggl_2.csv', sep=',', encoding='utf-8', index=False)


In [ ]:
import numpy as np
YOUR_API_KEY = 'xx'

df = pd.read_csv(f'{pathway}/files/combined.csv')

condition = []
# for index, row in df.iterrows():
#     print(index)
#     PLACE_ID = row['place']
place_list = ('ChIJ8RRrJ-YddkgR28mStkbXE3U', 'ChIJz2raGbUddkgR1EBfTLADLtI')
df_conditions = pd.DataFrame(columns=['status','database','api','identity'])

for pl in api_list:
# for index, row in df.iterrows():
#     PLACE_ID = row['place']
    PLACE_ID = 'ChIJo1EeGKkadkgRF6qptUS7DjI'
    PLACE_ID = pl
    print(pl)
    url = f"""https://maps.googleapis.com/maps/api/place/details/json?fields=website%2Curl%2Cname%2Ceditorial_summary&place_id={PLACE_ID}&key={YOUR_API_KEY}"""
    response = requests.get(url).json()
    # print(response)

    if 'editorial_summary' in response['result']:
        api_att = response['result']['editorial_summary']['overview']
        status = 'edit'
    #         if row[att] == '':
    #             df.loc[df['pub_identity'] == row['pub_identity'], 'extra'] = response['result']['editorial_summary']['overview']
    #             status = 'UPDATE'
    #         else:
    #             if row[att] == response['result']['editorial_summary']['overview']:
    #                 status = 'NO CHANGE'
    #             else:
    #                 status = 'DIFFERENCE'

    else:
        api_att = 'none'
        status = 'none'
    condition = [status, row['extra'], api_att, PLACE_ID]
    df_conditions_temp = pd.DataFrame([condition], columns=['status','database','api','place'])

    df_conditions = pd.concat([df_conditions, df_conditions_temp], ignore_index=True, sort=False)
    
#     df_conditions = pd.concat(df_conditions, df_conditions_temp)
# print(api_att)        
df_conditions

